# HW05: Double ML with XGBoost
(due October 27th)

In [29]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r10/nlswork.dta')
print(df.shape)
df = df.dropna()
print(df.shape)

(28534, 21)
(13452, 21)


In [30]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [31]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'race', 'msp','nev_mar', 'collgrad', 'not_smsa', 'c_city', 'south', 'ind_code', 'ttl_exp' ]]

In [32]:
from sklearn.model_selection import train_test_split
# split into sample A and sample B
X_A, X_B, Y_A, Y_B, D_A, D_B = train_test_split(X, Y, D, test_size=0.50, random_state=0)

# Within each sample, make a validation set for xgboost early stopping
X_A_train, X_A_val, Y_A_train, Y_A_val, D_A_train, D_A_val = train_test_split(X_A, Y_A, D_A, test_size=0.33, random_state=0)
X_B_train, X_B_val, Y_B_train, Y_B_val, D_B_train, D_B_val = train_test_split(X_B, Y_B, D_B, test_size=0.33, random_state=0)

In [36]:
# Step 1. In both samples, use xgboost regressor to predict log wages (outcome Y)
# use early stopping.
from xgboost import XGBRegressor
xgbr_Y_A = XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 100)

xgbr_Y_A.fit(X_A_train, Y_A_train, eval_metric="rmse", eval_set=[(X_A_val, Y_A_val)], verbose=False)

xgbr_Y_B = XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 100)

xgbr_Y_B.fit(X_B_train, Y_B_train, eval_metric="rmse", eval_set=[(X_B_val, Y_B_val)], verbose=False)

[23:07:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[23:07:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[23:07:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[23:07:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [38]:
# Step 2. In both samples, use xgboost classifier to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier
xgbc_D_A = XGBClassifier(objective ='binary:logistic', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 100)

xgbc_D_A.fit(X_A_train, D_A_train, eval_metric="error", eval_set=[(X_A_val, D_A_val)], verbose=False)

xgbc_D_B = XGBClassifier(objective ='binary:logistic', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 100)

xgbc_D_B.fit(X_B_train, D_B_train, eval_metric="error", eval_set=[(X_B_val, D_B_val)], verbose=False)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [42]:
# Step 3. Cross-fitting: Form predictions in other sample.

# predict wages in sample A using model trained in sample B:
Y_hat_A = xgbr_Y_B.predict(X_A)

# vice versa:
Y_hat_B = xgbr_Y_A.predict(X_B)

    
# predict union status in sample A using model trained in sample B:
D_hat_A = xgbc_D_B.predict(X_A)
    
# vice versa:
D_hat_B = xgbc_D_B.predict(X_A)
    

In [43]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
y_tilde_A =  Y_A - Y_hat_A
y_tilde_B =  Y_B - Y_hat_B

# residualized union status in samples A and B:
D_tilde_A =  D_A - D_hat_A
D_tilde_B =  D_B - D_hat_B

In [44]:
# as a shortcut, use statsmodels.
# for projects (not homework), we would do this in stata.
!pip install statsmodels

In [49]:
# Step 4. Run OLS regressions and produce DML estimate

import statsmodels.api as sm

model_A = sm.OLS(y_tilde_A, D_tilde_A)
result_A = model_A.fit(cov_type='HC1')
beta_A = result_A.params
se_A = result_A.bse

model_B = sm.OLS(y_tilde_B, D_tilde_B)
result_B = model_B.fit(cov_type='HC1')
beta_B = result_B.params
se_B = result_B.bse

beta = .5 * (beta_A + beta_B)
se = .5 * (se_A + se_B)

print('DML Coeff:',beta)
print('DML S.E.:', se)

DML Coeff: union    0.084783
dtype: float64
DML S.E.: union    0.00845
dtype: float64


In [52]:
import os

os.system('jupyter nbconvert --to html HW05.ipynb')

0